<a href="https://colab.research.google.com/github/Ajay-2007/deep-learning-projects/blob/master/1_feed_forward_neural_network_predicting_diabetes_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler  

In [0]:
data = pd.read_csv('diabetes.csv') 

In [22]:
data.head()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive


In [0]:
x = data.iloc[:, 0:-1]

In [58]:
x.head()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age
0,6,148,72,35,0,33.6,50
1,1,85,66,29,0,26.6,31
2,8,183,64,0,0,23.3,32
3,1,89,66,23,94,28.1,21
4,0,137,40,35,168,43.1,33


In [0]:
y = data.iloc[:,-1]

In [60]:
y.head()

0    positive
1    negative
2    positive
3    negative
4    positive
Name: Class, dtype: object

In [0]:
x = x.values

In [0]:
y = y.map({'positive' : 1, 'negative' : 0})

In [0]:
y = list(y)

In [0]:
y = np.array(y, dtype='float64')

In [65]:
type(x)

numpy.ndarray

In [66]:
x

array([[  6. , 148. ,  72. , ...,   0. ,  33.6,  50. ],
       [  1. ,  85. ,  66. , ...,   0. ,  26.6,  31. ],
       [  8. , 183. ,  64. , ...,   0. ,  23.3,  32. ],
       ...,
       [  5. , 121. ,  72. , ..., 112. ,  26.2,  30. ],
       [  1. , 126. ,  60. , ...,   0. ,  30.1,  47. ],
       [  1. ,  93. ,  70. , ...,   0. ,  30.4,  23. ]])

In [67]:
type(y[0])

numpy.float64

In [0]:
sc = StandardScaler()
x = sc.fit_transform(x) 

In [0]:
x = torch.tensor(x) 
y = torch.tensor(y).unsqueeze(1)

In [72]:
print(x.shape)
print(y.shape)

torch.Size([768, 7])
torch.Size([768, 1])


In [0]:
class Dataset(Dataset):

  def __init__(self, x, y):
    self.x = x
    self.y = y

  def __getitem__(self, index):
    return self.x[index], self.y[index]
  
  def __len__(self):
    return len(self.x)   

In [0]:
datasets = Dataset(x, y)

In [76]:
len(datasets)

768

In [0]:
train_loader = torch.utils.data.DataLoader(dataset = datasets,
                            batch_size = 32, 
                            shuffle = True)

In [79]:
train_loader

In [81]:
print(f'There is {len(train_loader)} batches in the dataset')

for x, y in train_loader:
  print(f'Data: {x.shape}')
  print(f'Labels: {y.shape}')
  break

There is 24 batches in the dataset
Data: torch.Size([32, 7])
Labels: torch.Size([32, 1])


In [82]:
768/32

24.0

In [0]:
class Model(nn.Module):
  def __init__(self, input_features, output_features):
    super(Model, self).__init__()
    self.fc1 = nn.Linear(input_features, 5)
    self.fc2 = nn.Linear(5, 4)
    self.fc3 = nn.Linear(4, 3)
    self.fc4 = nn.Linear(3, output_features)
    self.sigmoid = nn.Sigmoid()
    self.tanh = nn.Tanh()
  
  def forward(self, x):
    out = self.fc1(x)
    out = self.tanh(out)
    out = self.fc2(out)
    out = self.tanh(out)
    out = self.fc3(out)
    out = self.tanh(out)
    out = self.fc4(out)
    out = self.sigmoid(out)
    return out

In [0]:
net = Model(7, 1)
# criterion = torch.nn.BCELoss(size_average=True)
criterion = torch.nn.BCELoss(reduction='mean')
optimizer = torch.optim.SGD(net.parameters(), lr = 0.1, momentum=0.9)

In [92]:
epochs = 200
for epoch in range(epochs):
  for inputs, labels in train_loader:
    inputs = inputs.float()
    labels = labels.float()
    # Forward Propogation
    # outputs = net(inputs)
    outputs = net.forward(inputs)
    # Loss Calculation
    loss = criterion(outputs, labels)
    # Clear the gradient buffer (w <-- w - lr*gradient)
    optimizer.zero_grad()
    # Back Propogation
    loss.backward()
    # update Weights
    # w <-- w - lr*gradient    
    optimizer.step()

  # Accuracy Calculation
  output = (outputs > 0.5).float()
  # (output == labels).sum() / output.shape[0]
  accuracy = (output == labels).float().mean()
  # Print Statistics
  print('Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}'.format(epoch+1, epochs, loss, accuracy))

Epoch 1/200, Loss: 0.266, Accuracy: 0.875
Epoch 2/200, Loss: 0.321, Accuracy: 0.812
Epoch 3/200, Loss: 0.379, Accuracy: 0.844
Epoch 4/200, Loss: 0.562, Accuracy: 0.719
Epoch 5/200, Loss: 0.477, Accuracy: 0.719
Epoch 6/200, Loss: 0.364, Accuracy: 0.750
Epoch 7/200, Loss: 0.484, Accuracy: 0.688
Epoch 8/200, Loss: 0.326, Accuracy: 0.844
Epoch 9/200, Loss: 0.691, Accuracy: 0.594
Epoch 10/200, Loss: 0.548, Accuracy: 0.781
Epoch 11/200, Loss: 0.534, Accuracy: 0.719
Epoch 12/200, Loss: 0.377, Accuracy: 0.781
Epoch 13/200, Loss: 0.384, Accuracy: 0.812
Epoch 14/200, Loss: 0.364, Accuracy: 0.781
Epoch 15/200, Loss: 0.345, Accuracy: 0.812
Epoch 16/200, Loss: 0.478, Accuracy: 0.688
Epoch 17/200, Loss: 0.503, Accuracy: 0.781
Epoch 18/200, Loss: 0.342, Accuracy: 0.844
Epoch 19/200, Loss: 0.296, Accuracy: 0.906
Epoch 20/200, Loss: 0.385, Accuracy: 0.750
Epoch 21/200, Loss: 0.470, Accuracy: 0.812
Epoch 22/200, Loss: 0.554, Accuracy: 0.625
Epoch 23/200, Loss: 0.430, Accuracy: 0.844
Epoch 24/200, Loss: 